In [1]:
import pandas as pd
import numpy as np
from numpy.core import numerictypes
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [2]:
def pluralize_gift(gift):
  if 'oo' in gift:
    gift = gift.replace('oo', 'ee') #conditions for the endings of strings
  elif gift[-1] == 'y':
    gift = gift.replace('y', 'ies')
  else:
    gift += 's'
  return gift

In [3]:
pluralize_gift = np.vectorize(pluralize_gift) #vectorizing

In [4]:
# Should work
print(pluralize_gift("Goose"))
print(pluralize_gift(xmas['Gift.Item']))

Geese
['partridges' 'doves' 'hens' 'birds' 'rings' 'geese' 'swans' 'maids'
 'ladies' 'lords' 'pipers' 'drummers']


In [5]:
def add_whitespace(string): #adding whitespace if there is a value else returns value.
    if string:
        return ' ' + string
    else:
        return string

In [6]:
def make_phrase(num, num_word, verb, item, adjective, location):
  verb = verb.fillna('')  #fillna detects na and fills value with empty string
  verb = verb.str.strip() #strip removes whitespace start and end of string
  adjective = adjective.fillna('')
  adjective = adjective.str.strip()
  location = location.fillna('')
  location = location.str.strip()
  item = item.str.strip()

  more_than_one_day = num > 1
  first_day = num == 1
  vowels = ('a', 'e', 'i', 'o', 'u')

  number_mappings = {1: 'a',2: 'two', 3: 'three', 4: 'four',
                      5: 'five', 6: 'six', 7: 'seven',
                      8: 'eight', 9: 'nine', 10: 'ten',
                      11: 'eleven', 12: 'twelve'} #mapping for each number
  num = num.map(number_mappings)
  first_day_str = num == 'a'
  item[more_than_one_day] = item.apply(pluralize_gift)
  item_starting_with_vowels = item.str.startswith(vowels)
  num[first_day_str & item_starting_with_vowels] = 'an'
  verb = verb.apply(lambda x: add_whitespace(x)) #checking whitespaces for verb column
  adjective = adjective.apply(lambda x: add_whitespace(x)) #checking whitespaces for adjective column
  item = item.apply(lambda x: add_whitespace(x)) #checking whitespaces for item column
  location = location.apply(lambda x: add_whitespace(x)) #checking whitespaces for location column
  full_phrase = num + adjective + item + verb + location


  return num, num_word, verb, item,  adjective, location, full_phrase #returns phrase in order

In [7]:
def sing_day(dataset, num, phrase_col):
  """
  <documentation>
  """
  num_word = dataset.loc[dataset['Day']== num, 'Day.in.Words'][num-1] #getting the data in column Day.in.Words from the row where the column day has the value of num
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
  gifts = []
  if num == 1:
    gifts.append("\n"+ dataset[phrase_col][0].strip()+ ".")
    return intro +"".join(gifts)
  for i in range(num-1, -1, -1): #conditions on how to append for each condition with mapping
    if i == num-1:
      gifts.append("\n"+ dataset[phrase_col][i].strip()+ ",\n")
    elif i > 0 and i < num-1:
      gifts.append(dataset[phrase_col][i].strip() + ",\n")
    elif i == 0:
      gifts.append("and "+dataset[phrase_col][i].strip()+".")

  return intro +"".join(gifts)

In [8]:
xmas['Day1'], xmas['Day.in.Words'], xmas['Verb'], xmas['Gift.Item'],  xmas['Adjective'], xmas['Location'], xmas['Full.Phrase'] = make_phrase(xmas['Day'], xmas['Day.in.Words'], xmas['Verb'], xmas['Gift.Item'],  xmas['Adjective'], xmas['Location'])

In [9]:
print(sing_day(xmas, 3, "Full.Phrase")) #testing one condition

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


In [10]:
result = list(map(lambda num: sing_day(xmas, num, "Full.Phrase"), [1,2,3,4,5,6,7,8,9,10,11,12] ))
print("\n\n".join(result))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

In [11]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [12]:
xmas2['Day1'], xmas2['Day.in.Words'], xmas2['Verb'], xmas2['Gift.Item'], xmas2['Adjective'], xmas2['Location'], xmas2['Full.Phrase'] = make_phrase(xmas2['Day'], xmas2['Day.in.Words'], xmas2['Verb'], xmas2['Gift.Item'],  xmas2['Adjective'], xmas2['Location'])
xmas2 #checking out new dataset and appending using previous functions

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day1,Full.Phrase
0,1,first,email,,,from Cal Poly,an,an email from Cal Poly
1,2,second,points,,meal,,two,two meal points
2,3,third,pens,,lost,,three,three lost pens
3,4,fourth,reviews,,course,,four,four course reviews
4,5,fifth,exams,,practice,,five,five practice exams
5,6,sixth,graders,grading,,,six,six graders grading
6,7,seventh,seniors,stressing,,,seven,seven seniors stressing
7,8,eighth,moms,a-calling,,,eight,eight moms a-calling
8,9,ninth,parties,bumping,,,nine,nine parties bumping
9,10,tenth,loads,of laundry,,,ten,ten loads of laundry


In [13]:
print(sing_day(xmas2, 1, "Full.Phrase")) #printing new song

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.


In [14]:
result = list(map(lambda num: sing_day(xmas2, num, "Full.Phrase"), [1,2,3,4,5,6,7,8,9,10,11,12] ))
print("\n\n".join(result))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t